In [1]:
from PIL import Image
import pytesseract
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
import time
import cv2
import gradio as gr

pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

In [2]:
import pandas as pd
from glob import glob
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import re
import seaborn as sn

from tensorflow.keras.layers import Input,Dense,Bidirectional,Conv2D,MaxPooling2D,Flatten,concatenate,GlobalAveragePooling2D,BatchNormalization,Lambda,Add,Multiply
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam,SGD
import tensorflow.keras.backend as K

from transformers import AutoTokenizer, TFAutoModel, TFBertModel, logging

from utilities import *

logging.set_verbosity_error()

In [3]:
def class_block(inputs):
    X = tf.keras.layers.BatchNormalization()(inputs)
    X = tf.keras.layers.Dense(512, activation='relu')(X)
    X = tf.keras.layers.Dropout(0.2)(X)
    X = tf.keras.layers.Dense(128, activation='relu')(X)
    X = tf.keras.layers.Dropout(0.2)(X)
    
    y = tf.keras.layers.Dense(3, activation='softmax', name='outputs')(X)  # 3 labels due to three sentiment classes

    return y

In [4]:
seq_len=50
seq_len2=10

model=  "unideeplearning/polibert_sa"
tokenizer = AutoTokenizer.from_pretrained(model)
bert = TFBertModel.from_pretrained(model)

CNN = tf.keras.applications.ResNet50V2(include_top=False, weights='imagenet', input_tensor=None, input_shape=(224,224,3), pooling=False, classes=3)

In [5]:
input_ids = tf.keras.layers.Input(shape=(seq_len,), name='input_ids', dtype='int32')
mask = tf.keras.layers.Input(shape=(seq_len,), name='attention_mask', dtype='int32')
input_ids2 = tf.keras.layers.Input(shape=(seq_len2,), name='input_ids2', dtype='int32')
mask2 = tf.keras.layers.Input(shape=(seq_len2,), name='attention_mask2', dtype='int32')

input_ids_c=tf.keras.layers.concatenate([input_ids, input_ids2])
mask_c=tf.keras.layers.concatenate([mask, mask2])

image_inputs=tf.keras.layers.Input(shape=(224,224,3),name="images")

text_embeddings = bert(input_ids_c, attention_mask=mask_c)[0]  # we only keep tensor 0 (last_hidden_state)
text_1d = tf.keras.layers.GlobalMaxPool1D()(text_embeddings)  # reduce tensor dimensionality

image_embeddings=CNN(image_inputs)
image_1d=GlobalAveragePooling2D()(image_embeddings)

X=tf.keras.layers.concatenate([text_1d, image_1d])

y=class_block(X)

model = tf.keras.Model(inputs=[input_ids, mask,input_ids2, mask2, image_inputs], outputs=y)

# freeze the DistilBERT layer
model.layers[7].trainable = False
model.layers[8].trainable = False

In [6]:
model.load_weights('saves/final_weights')

In [7]:
def preprocess_finale(im):
    im= cv2.bilateralFilter(im,5, 55,60)
    im = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
    _, im = cv2.threshold(im, 240, 255, 1)
    return im

In [8]:
custom_config = r"--oem 3 --psm 11 -c tessedit_char_whitelist='ABCDEFGHIJKLMNOPQRSTUVWXYZ '"
labels=['negative', 'neutral', 'positive']

def ext_text(tweet_text, tweet_image):
    text1=tweet_text
    im= tweet_image
    
    img1 = preprocess_finale(im)
    text = pytesseract.image_to_string(img1, lang='ita', config=custom_config)
    text = text.replace('\n', '')
    
    toktx=tokenize(text1, tokenizer, SEQ_LEN=50)
    toktx2=tokenize(text, tokenizer, SEQ_LEN=10)
    image = preprocess_image(im,labels=None, prediction=True)
    prediction= model.predict([toktx,toktx2,image])
    
    return img1, text, {labels[i]: float(prediction[0][i]) for i in range(0,3)}

In [9]:
iface = gr.Interface(fn=ext_text, inputs=[gr.inputs.Textbox(), gr.inputs.Image()], 
                     outputs=[gr.outputs.Image(label='Preprocessed image'),                               
                              gr.outputs.Textbox(label='Extracted Text'), 
                              gr.outputs.Label(num_top_classes=3, label='Predicted sentiment')],
                     examples=[('', 'examples/example'+str(i)+'.png') for i in range(1,6)]
                    )
iface.launch(share=True)

Running locally at: http://127.0.0.1:7860/
This share link will expire in 24 hours. If you need a permanent link, email support@gradio.app
Running on External URL: https://38010.gradio.app
Interface loading below...


(<Flask 'gradio.networking'>,
 'http://127.0.0.1:7860/',
 'https://38010.gradio.app')